In [152]:
# Подключение модулей
from keras.models import Sequential
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
import cv2, numpy as np

In [153]:
# Подключение Google Диска
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [154]:
# Пути к наборам данных
train_dir = '/content/drive/MyDrive/horses_or_humans/train'
test_dir = '/content/drive/MyDrive/horses_or_humans/val' 
# Размер изображений
img_width, img_height = 300, 300
# Размер мини-выборки
batch_size = 64
# Кол-во изображений для обучения
nb_train_samples = 1027
# Кол-во изображений для теста
nb_test_samples = 256

In [155]:
# Создание генератора изображений
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [156]:
# Создание экземпляра модели сети VGG16
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
# weights - веса предварительно обученной сети
# include_top = false означает, что мы загружаем только сверточную часть сети, без квалификационной
# input_shape - размер тензора

# Сверточную часть сети обучать не надо
vgg16_net.trainable = False

vgg16_net.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 300, 300, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0     

In [157]:
# Создание модели составной сети
model = Sequential()
# Добавляем сверточные слои
model.add(vgg16_net)
# Преобразуем двумерный массив vgg16 в одномерный
model.add(Flatten())
# Полносвязный слой
model.add(Dense(256, activation='relu'))
# Слой регуляризации (для предотвращения переобучения)
model.add(Dropout(0.5))
# Кол-во классов
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 9, 9, 512)         14714688  
_________________________________________________________________
flatten_22 (Flatten)         (None, 41472)             0         
_________________________________________________________________
dense_53 (Dense)             (None, 256)               10617088  
_________________________________________________________________
dropout_31 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_54 (Dense)             (None, 1)                 257       
Total params: 25,332,033
Trainable params: 10,617,345
Non-trainable params: 14,714,688
_________________________________________________________________


In [158]:
# Компилируем составную сеть
model.compile(loss='binary_crossentropy',
              optimizer='SGD', 
              metrics=['accuracy'])

In [159]:
# Обучение составной сети
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=5,
    validation_data=test_generator,
    validation_steps=nb_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
16/16 [==============================] - 608s 38s/step - loss: 1.5034 - accuracy: 0.5523 - val_loss: 0.4848 - val_accuracy: 0.8633
Epoch 2/5
16/16 [==============================] - 606s 40s/step - loss: 0.5968 - accuracy: 0.6979 - val_loss: 0.3318 - val_accuracy: 0.9414
Epoch 3/5
16/16 [==============================] - 606s 38s/step - loss: 0.3894 - accuracy: 0.8296 - val_loss: 0.5245 - val_accuracy: 0.5430
Epoch 4/5
16/16 [==============================] - 605s 38s/step - loss: 0.4207 - accuracy: 0.7972 - val_loss: 0.2339 - val_accuracy: 0.9062
Epoch 5/5
16/16 [==============================] - 606s 38s/step - loss: 0.2181 - accuracy: 0.9369 - val_loss: 0.2242 - val_accuracy: 0.9023


In [160]:
# Проверяем модель на тестовой выборке
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print('Точность работы на тестовых данных: %.2f%%' % (scores[1]*100))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Точность работы на тестовых данных: 90.23%
